In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# FFT and Ewald Summation

The Fast Fourier Transform (FFT) is a method that is used frequently in the calculation of certain terms in the simulation of material, as well as being very important in signal and image processing. It is used to compute the discrete Fourier transform of a sequence or its inverse.

## Discrete Fourier Transform

The discrete Fourier transform of a sequence of $N$ numbers $x_0$, $x_1$, $\dots$, $x_{N-1}$ produces a corresponding sequence $X_0$, $X_1$, $\dots$, $X_{N-1}$ given by

$$ X_k = \Sigma_{j=0}^{N-1} x_j \mathrm{e}^{-i 2 \pi j k/N}.$$

The inverse transform is then given by

$$ x_j = \frac{1}{N}\Sigma_{k=0}^{N-1} X_k \mathrm{e}^{i 2 \pi j k/N}.$$

Note - this pair of transformations are often presented slightly differently. In particular how they are normalized and the signs of the exponents are chosen by convention. As long as both exponents have opposite sign, and the product of the normalization factors is $1/N$, any choice is fine. It's also common to see both transformations having a $1/\sqrt{N}$ normalization factor for example.

For example, let's say $x_j$ represents the sampled position of an atom every timestep $\tau$. We could use the discrete Fourier transform to find the frequencies with which the atom is vibrating. Each point of the transform $X_k$ will correspond to a frequency $\frac{k}{\tau N}$.

In [ ]:
# Let's start with a function that generates some oscillating data using a sin
def vib_data1(num_points, freq, phase, timestep):
    '''Generate N points from a given sine wave'''
    # Initialize an array of the correct size, and assign the correct
    # value to each element in a loop.
    samples = np.empty(num_points)
    for it in range(num_points):
        samples[it] = np.sin(2 * np.pi * freq * timestep * it + phase)
    return samples

# Here's another small diversion looking at how we might make things fast with numpy.

def vib_data2(num_points, freq, phase, timestep):
    '''Generate N points from a given sine wave'''
    # Use a list comprehension to generate a list of all the arguments we want to pass
    # to the sin function to generate our sampled data. Then use the numpy sin function
    # on this whole array, as it will be applied element-wise.
    sin_args = np.array([(2 * np.pi * freq * it * timestep + phase) for it in range(num_points)])
    return np.sin(sin_args)

# We could also do this in a single command. Which do you think is faster, and why?
def vib_data3(num_points, freq, phase, timestep):
    '''Generate N points from a given sine wave'''
    return np.array([np.sin(2 * np.pi * freq * it * timestep + phase) for it in range(num_points)])

# Another optimization we could make is to take the constant terms from sin_args
# and multiply them outside the list comprehension. The same can be done with the phase. Allowing us
# to switch to a full set of vector operations.
def vib_data4(num_points, freq, phase, timestep):
    '''Generate N points from a given sine wave'''
    arg_factor = 2 * np.pi * freq * timestep
    t_indices = np.arange(num_points) # This makes an array with numbers 0 to num_points-1
    sin_args = t_indices * arg_factor + phase
    return np.sin(sin_args)

# All these routines output the exact same thing for the same inputs.
# They all differ slightly in their implementation though.
# You can compare the speed of them using the %timeit ipython magic.

In [ ]:
# Now let's define a function for the forward transformation
# We're going to do this explicitly to start with so you can see it scales as O(N^2)
def forward_dft(xl):
    '''Forward discrete Fourier transform'''
    N = len(xl)
    Xk = np.zeros(N, dtype=complex)
    expfactor = -2j * np.pi / N
    # Let's do this explicitly to start with
    for k in range(N):
        for l in range(N):
            Xk[k] = Xk[k] + xl[l] * np.exp(expfactor * k * l)
    return Xk

In [ ]:
# Initialize and plot our sampled data
n = 20; f= 0.1; p = np.pi * 0; t = 1;
samples = vib_data1(num_points=n, freq=f, phase=p, timestep=t)
plt.plot(samples)
plt.show()

# And let's try our transform with the samples we generated
ft_samples = forward_dft(samples)
#print(ft_samples)
freqs = np.arange(len(samples)) / (t * len(samples))
# The output is complex so we plot the real and imaginary parts.
plt.plot(freqs, np.real(ft_samples), label="Real")
plt.plot(freqs, np.imag(ft_samples), label="Imaginary")
plt.legend()
plt.show()

So we have generated samples using a sine wave with frequency 0.1, and we get peaks in the imaginary part of the transform at 0.1 and 0.9.

Let's try the following, and see how it affects the transform each time
- change the input frequency to 0.3
    - You should now see peaks in the transform at 0.3 and 0.7
- change the phase to $\pi/2$ to make our sampled wave a cosine
    - This leads to a change of phase in the transformed signal also
- change the frequency to 0.11
    - The structure now looks quite different. This will happen for any frequency that's not a multiple of $\frac{1}{\tau N}$. For this reason it can be useful to instead look at the power spectrum, by calculating the absolute square at each transformed point. Try doing this now. Note - you can square a numpy complex array, but you'll need to take the real part to plot it as the output is still complex.
    
You may have noticed our transform peaks are at $f$ and $1-f$. But why are we getting two peaks rather than 1?

## Aliasing

In the first case, we got peaks in our transform at $f=0.1$ and $f=0.9$. Let's try generating samples at both of these frequencies and comparing them:

In [ ]:
samples_a = vib_data1(num_points=20, freq=0.1, phase=0.0, timestep=1.0)
samples_b = vib_data1(num_points=20, freq=0.9, phase=0.0, timestep=1.0)

plt.plot(samples_a, 'bo')
plt.plot(samples_b, 'r+')
plt.show()

The sampled data looks like it has the same frequency for both cases, but opposite phase! How did this happen?

This occurs due to a phenomenom called aliasing. If we decrease the sampling period (timestep), and increase the total number of samples correspondingly, we should be able to resolve these two sets of samples:

In [ ]:
samples_a = vib_data1(num_points=2000, freq=0.1, phase=0.0, timestep=0.01)
samples_b = vib_data1(num_points=2000, freq=0.9, phase=0.0, timestep=0.01)

times = np.arange(20, step=0.01) # Since our timestep isn't 1 we need to set the x-axis values.
plt.plot(times, samples_a, 'bo')
plt.plot(times, samples_b, 'r+')
plt.show()

Now it's clear these two curves are different, but for the sampling interval we had, they both return the same points (but with opposite phase).

Waves sampled in steps of $\tau$ thus have a limit as to how high a frequency we can resolve. This is given by
$$ f_{Ny} = \frac{1}{2\tau} $$
which is known as the Nyquist frequency.

So with $\tau=1$ we should truncate our transformed output at a frequency of 0.5. While it may seem we are throwing data away by doing this, our transform of N real samples generated N complex numbers. This means that the information in our transformed output must contain duplication as the amount of information can't have doubled.

## FFT

Look at the implementation of the discrete Fourier transform above. You can see that as we have a double nested loop, each with $N$ elements, the implementation will scale with O($N^2$).

The Fast Fourier Transform (FFT) algorithm is based on a clever reorganisation of the nested sum that improves the scaling to O($N\log N$). The method originally relied on $N$ being a power of 2 so that the summation could be repeatedly broken down into smaller parts. It has been steadily generalized since then, so that even prime values of $N$ can achieve O($N\log N$) scaling.

The FFT will produce exactly the same output as the naive implementation above, but allows significantly larger data sets to be used.

Modern implementations are fairly complex, but thankfully, libraries to perform the FFT are ubiqitous.

## NumPy

NumPy has a module dedicated to the discrete Fourier transform: [numpy.fft](https://docs.scipy.org/doc/numpy/reference/routines.fft.html). This has range of functions available depending on the dimensionality and other characteristics of the data you are analysing.

Let's use one with our samples as before. We'll use the [rfft()](https://docs.scipy.org/doc/numpy/reference/generated/numpy.fft.rfft.html) function which is written for real input, as all our sampled data is real.

In [ ]:
n = 20; f= 0.1; p = np.pi * 0; t = 1;
samples = vib_data1(num_points=n, freq=f, phase=p, timestep=t)
plt.plot(samples)
plt.show()

# We'll use the rfft function which is for real input
ft_samples = np.fft.rfft(samples)
# Note: the number of points returned by this function is
# (n/2)+1 for an even number of samples, and (n+1)/2 for an odd number
# due to the aliasing issue discussed earlier.

# To get the associated frequencies, this helper function can be used.
freqs = np.fft.rfftfreq(len(samples), t)

# The output is complex so we plot the real and imaginary parts.
plt.plot(freqs, np.real(ft_samples), label="Real")
plt.plot(freqs, np.imag(ft_samples), label="Imaginary")
plt.legend()
plt.show()

# And lets calculate the inverse of this to confirm we get our sampled data back
ift_samples = np.fft.irfft(ft_samples)
plt.plot(ift_samples)
plt.show()

# SciPy

There are also more detailed implementations available in the SciPy module [fftpack](https://docs.scipy.org/doc/scipy/reference/fftpack.html). This module offers a wider range of functions than the equivalent NumPy module, and can be a little better optimized also.

Where equivalent NumPy and SciPy functions are available, it's generally best to chose the SciPy version as these are the versions where development is focused. We could use the [rfft](https://docs.scipy.org/doc/scipy/reference/generated/scipy.fftpack.rfft.html) SciPy function in exactly the same way as the NumPy version used above, although the conventions for the output are a little different, with the SciPy version returning a real array containing the complex parts of the transform (which is honestly pretty awkward to work with - there has been some discussion about changing this, or making new scipy functions with numpy style output, so be sure to check the doc pages if you're referring to this notebook from the future).

In [ ]:
import scipy.fftpack
ft_samples = scipy.fftpack.rfft(samples)
freqs = scipy.fftpack.rfftfreq(len(ft_samples), t)

# The output is in a single array containing both the real and imaginary components
plt.plot(freqs, np.real(ft_samples), label="Real & Imaginary")
plt.legend()
plt.show()

# And lets calculate the inverse of this to confirm we get our sampled data back
ift_samples = scipy.fftpack.irfft(ft_samples)
plt.plot(ift_samples)
plt.show()

# Compare the speed of both functions
%timeit np.fft.rfft(samples)
%timeit scipy.fftpack.rfft(samples)

## Mutidimensional Discrete Fourier Transform

One of the main uses of the discrete Fourier transform you will come across is in the transformation of quantities such as electronic charge density from real space to reciprocal (wavevector) space in calculations where periodic boundary conditions are used. This is a three-dimensional problem rather than the one dimensional problem we've looked at so far.

The expression for the three dimensional discrete Fourier transform is given by nesting three one-dimensional transformations as follows:
$$ X_{k_1, k_2, k_3} = \Sigma_{j_1=0}^{N_1-1} \left[\mathrm{e}^{-i 2 \pi j_1 k_1/N_1} \Sigma_{j_2=0}^{N_2-1} \left(\mathrm{e}^{-i 2 \pi j_2 k_2/N_2} \Sigma_{j_3=0}^{N_3-1} x_j \mathrm{e}^{-i 2 \pi j_3 k_3/N_3}\right)\right].$$

This is more usually written in terms of vectors $\mathbf{k}$ and $\mathbf{j}$ as
$$ X_\mathbf k = \Sigma_{\mathbf j=\mathbf 0}^{\mathbf N-1} x_\mathbf j \mathrm{e}^{-i 2 \pi \mathbf j\cdot (\mathbf k/\mathbf N)},$$

where $\mathbf N -1 = (N_1-1, N_2-1, N_3-1)$ and $\mathbf k /\mathbf N = (k_1/N_1, k_2/N_2, k_3/N_3)$.

There is both a [numpy.fft.fftn](https://docs.scipy.org/doc/numpy/reference/generated/numpy.fft.fftn.html) and a [scipy.fftpack.fftn](https://docs.scipy.org/doc/scipy/reference/generated/scipy.fftpack.fftn.html) function which can be used to calculate n-dimensional FFT. Note there are also two dimensional versions if you are working with a 2D model.

## Uses in Materials Simulations

### The Hartree term

If we have some distribution of electrons $n(r)$ in a periodic system, it's common to calculate the their electrostatic interaction with each other as a single term contributing to the total energy of the system (along with several other terms depending on what method is being used). This is usually referred to as the Hartree term and is given by:
$$ E_\mathrm H = \frac{1}{2} \int \int \frac{n(\mathbf r) n(\mathbf r')}{|\mathbf r-\mathbf r'|}
\mathbf d\mathbf r \mathbf d\mathbf r' $$

As you might imagine, this term when considered in isolation, diverges since it only considers the negative electrons and  we have a periodic (infinitely repeating) system. You can see this if you consider a one dimensional system consisting of a chain of equal charges, $q$, per cell of dimension $a$, and each charge at position $x$ within its cell. The energy of these charges interacting with each other if we consider $N$ neighbouring cells is given by
$$ E_e = \frac{1}{2} \Sigma_{i=1}^N \Sigma_{j=1,j\ne i}^N \frac{q^2}{|x_i - x_j|}
= N\left(\frac{q^2}{2a}\right)\Sigma_{i=1}^N \frac{1}{i} $$
which diverges as $N$ is increased.

Fortunately, the diverging term is cancelled by similar terms in the ion-ion and electron-ion terms when all three are taken together. We can then calculate the Hartree term efficiently with an FFT as
$$ E_\mathrm H = \frac{1}{2} \int \int \frac{n(\mathbf r) n(\mathbf r')}{|\mathbf r-\mathbf r'|}
\mathrm d\mathbf r \mathrm d\mathbf r'
= \frac{2\pi}{V}\Sigma_{\mathbf G\ne 0} \frac{|\tilde n(\mathbf G)|^2}{G^2}$$
where $V$ is the cell volume and $\tilde n(\mathbf G)$ is reciprocal space charge distribution, given by the FFT of the real space charge distribution $n(\mathbf r)$. The $G=0$ term is infinite and corresponds to the divergent part of the Hartree term that would be cancelled by other contributions to the energy as long as the system is charge neutral overall, so we can omit it here. So we can replace a fairly intensive double integral over real space, with a sum over FFT coefficients.

## Ewald Summation

Another important application of the FFT in the simulation of materials is in Ewald summation. The problem is quite similar to that outlined in the case of the Hartree term above. This time we have some grid of point charges. This would be the case for the nuclei in any calculation where we use the Born-Oppenheimer approximation and treat the nuclei as fixed points and solve for the electrons - we still need to add in a term that represent the interaction of these positive nuclei with each other at the end. This electrostatic contribution to the total energy of a set of ions with charge $Z_i$ and position $\mathbf R_i$ will be given by
$$ E_I = \frac{1}{2} \Sigma_{i, j, i\ne j} \frac{Z_i Z_j}{|\mathbf R_i -\mathbf R_j|} $$

We could think of our set of point charges as represented by delta function distributions and write this in a similar way as we the Hartree energy term. However, the Fourier transform of a delta function is completely delocalized, so trying to sum over our Fourier coefficients will also give us convergence problems.

The way to get around this is to break up this slowly converging sum into two parts, one of which is done in real space, with the other in reciprocal space, both of which are well behaved and converge rapidly.
- In real space we have our original lattice of charges, and around each we put some smooth and smeared out distribution of opposite charge, such as a Gaussian (typically Error functions are used rather than Gaussians in practice). The energy of this charge distribution will converge rapidly as at long range the oppositely charged Gaussian distributions will exactly cancel the delta functions due to the point charges. So this is short ranged in real space.
- Now we have another distribution of Gaussians that we need to add to counteract the set we have added. These are spread out in real space, which means they will be localized in reciprocal space. So we can handle these in the same way as the Hartree term described earlier.

In this way, the FFT can be used to calculate this term very rapidly.

In practice, whenever you need to integrate over something that's very delocalized in real space, consider whether it may be beneficial to calculate the Fourier transform and perform the integral in reciprocal space.